In [1]:
# 필요한 라이브러리 추가
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import logging
from typing import List
from transformers import pipeline

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 텍스트 파일 경로 설정
text_file_path = 'data/Q32.txt'

# 1. 데이터 로드
def load_sentences(file_path: str) -> List[str]:
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            sentences = file.readlines()
        # 각 문장의 양쪽 공백 제거 및 빈 문장 제거
        sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
        logging.info(f"{len(sentences)}개의 문장을 로드했습니다.")
        return sentences
    except FileNotFoundError:
        logging.error(f"파일을 찾을 수 없습니다: {file_path}")
        return []

sentences = load_sentences(text_file_path)

# 2. 분류 기준 정의
categories = [
    "1. 가격/물가 개선",
    "2. 교통환경 개선(교통수단, 인프라등)",
    "3. 더 많은 홍보/행사 개최",
    "4. 체험시설 및 관광상품 다양화",
    "5. 숙박시설 개선/확충",
    "6. 볼거리/즐길거리 다양화",
    "7. 먹거리 다양화",
    "8. 서비스 질 개선(안내, 친절도등)",
    "9. 편의시설 개선(화장실등)",
    "10. 관광 인프라 구축",
    "11. 청결도, 위생관리 필요",
    "12. 기타"
]

# 3. 로컬 분류 모델 설정
# 분류할 카테고리가 정해져 있으므로 다중 클래스 분류 모델을 사용할 수 있습니다.
# 사전 학습된 모델을 로드하고 분류 파이프라인을 설정합니다.
# 필요한 경우 미세 조정된 모델을 사용하세요.

# 예시: Zero-shot 분류를 사용하여 사전 정의된 카테고리에 분류
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# 4. 문장 분류 함수 작성
def classify_sentence_local(sentence: str) -> str:
    """
    주어진 문장을 지정된 카테고리 중 하나로 분류합니다.

    Args:
        sentence (str): 분류할 문장.

    Returns:
        str: 분류 결과 ('번호: 카테고리 이름') 또는 '오류'
    """
    try:
        result = classifier(sentence, categories, multi_label=False)
        # 가장 높은 점수를 받은 카테고리 선택
        top_category = result['labels'][0]
        category_index = categories.index(top_category) + 1  # 1부터 시작
        classification = f"{category_index}: {top_category.split('. ', 1)[1]}"
        logging.info(f"문장 분류 성공: {classification}")
        return classification
    except Exception as e:
        logging.error(f"분류 생성 중 오류 발생: {e}")
        return "오류"

# 5. 모든 문장 분류
def classify_sentences_local(sentences: List[str]) -> List[dict]:
    """
    여러 문장을 분류하여 리스트로 반환합니다.

    Args:
        sentences (List[str]): 분류할 문장들의 리스트.

    Returns:
        List[dict]: 분류된 데이터의 리스트.
    """
    classified_data = []
    total = len(sentences)
    for idx, sentence in enumerate(sentences, 1):
        classification = classify_sentence_local(sentence)
        classified_data.append({
            '문장': sentence,
            '분류': classification
        })
        print(f"{idx}/{total}: {classification} - {sentence}")
    return classified_data

classified_data = classify_sentences_local(sentences)

# 6. 저장 폴더 경로 설정
save_folder = os.path.join('data', os.path.splitext(os.path.basename(text_file_path))[0])

# 폴더가 없는 경우 생성
os.makedirs(save_folder, exist_ok=True)

# 7. 한글 폰트 설정
font_path = 'C:/Windows/Fonts/malgun.ttf'  # 시스템에 맞게 변경
try:
    fontprop = fm.FontProperties(fname=font_path)
    plt.rc('font', family='Malgun Gothic')
    logging.info("한글 폰트 설정 완료.")
except FileNotFoundError:
    logging.warning(f"폰트 파일을 찾을 수 없습니다: {font_path}")
    # 기본 폰트로 설정하거나 다른 폰트 경로를 지정할 수 있습니다.

# 8. 엑셀 파일 경로 설정
excel_file_path = os.path.join(save_folder, 'classification_results.xlsx')

# 9. 엑셀 파일로 저장
df = pd.DataFrame(classified_data)
df.to_excel(excel_file_path, index=False, header=True)
logging.info(f"Classification results saved to {excel_file_path}")

print(f"\nClassification results saved to {excel_file_path}")


2024-09-22 15:45:19,367 - INFO - 353개의 문장을 로드했습니다.


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

c:\Users\user\.conda\envs\langchain_gpu_11.7.1_8.5.0_python3.9.19\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--facebook--bart-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

352/353: 3: 더 많은 홍보/행사 개최 - 활발한 홍보활동을 부탁드립니다


2024-09-22 16:04:55,699 - INFO - 문장 분류 성공: 5: 숙박시설 개선/확충
2024-09-22 16:04:55,701 - INFO - 한글 폰트 설정 완료.


353/353: 5: 숙박시설 개선/확충 - 휴게소 설치


2024-09-22 16:04:55,910 - INFO - Classification results saved to data\Q32\classification_results.xlsx



Classification results saved to data\Q32\classification_results.xlsx
